## El procedimiento:

- Análisis y limpieza de los datos
- Construir nuestro modelo de predicción 
- Evaluar el modelo y calidad de predicción

In [1]:
# Leer y manipular los datos

import numpy as np
import pandas as pd

In [2]:
# librerias para machine learning

import sklearn
from sklearn.model_selection import train_test_split

In [4]:
# librerías para las gráficas

import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")

In [5]:
# los datos se cargan en variable

df_original = pd.read_csv("titanic.csv")

In [7]:
# Ver la información
df_original.head(5) # df == data frame

,PassengerId,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [10]:
# drop algunas columnas -> según las características que no son relevantes

df_dropped_features = df_original.drop(["PassengerId", "Name"], axis = 1)

df_dropped_features.head(5)

,Survived,Pclass,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [11]:
# vemos los diferentes valores del feature sex

print("sexo", df_dropped_features["Sex"].unique())

sexo ['male' 'female']


In [13]:
# con get_dumies de panda se categoriza los diferentes features
df_categorized = pd.get_dummies(df_dropped_features)

df_categorized.head(5)

,Survived,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Sex_female,Sex_male
0,0,3,22.0,1,0,7.2500,0,1
1,1,1,38.0,1,0,71.2833,1,0
2,1,3,26.0,0,0,7.9250,1,0
3,1,1,35.0,1,0,53.1000,1,0
4,0,3,35.0,0,0,8.0500,0,1


In [14]:
df_categorized.describe()

,Survived,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Sex_female,Sex_male
count,887.000000,887.000000,887.000000,887.000000,887.000000,887.00000,887.000000,887.000000
mean,0.385569,2.305524,29.471443,0.525366,0.383315,32.30542,0.354002,0.645998
std,0.487004,0.836662,14.121908,1.104669,0.807466,49.78204,0.478480,0.478480
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,0.000000,2.000000,20.250000,0.000000,0.000000,7.92500,0.000000,0.000000
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.45420,0.000000,1.000000
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.13750,1.000000,1.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.32920,1.000000,1.000000


In [15]:
# copia de todo el dataset
df_cleaned = df_categorized.copy()

# adios filas con valores NAN (valores no asignados)

# df_cleaned["Age"] = df_categorized.dropna(axis=1)


In [ ]:
def plot_with_bar(values = None, title = None)